<a href="https://colab.research.google.com/github/umacchi/bert-book/blob/main/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# 4-1
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 95.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 34.1 MB/s eta 0:00:00
  Created wheel for fugashi: filename=fugashi-1.1.0-cp310-cp310-linux_x86_64.whl size=257501 sha256=d6026adae8138c864e49d06b6b5ffafc5fe7dd371b49b9f7454841d225f12f2a
  Stored in directory: /root/.cache/pip/wheels/f6/bc/9b/6d38a64c5bea6582a87574f5c63b6c0bd9a4f5a6706ed577a5
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=66b40e4e098bceb2e398b0d7d17fb8831c039b45f415fa65b409a42b8a93066e
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed

In [2]:
# 4-2
import torch
from transformers import BertJapaneseTokenizer, BertModel

In [3]:
# 4-3
# 学習済みのトークナイザを取得する
model_name = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# 今回使用するBERTの日本語のトークナイザは、以下の流れでトークン化を行う
# 1. 形態素解析：MeCabを用いて、単語を分割する
# 2. Wordpieceを用いて、単語をトークンに分割する

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [ ]:
# 4-4
tokenizer.tokenize('明日は自然言語処理の勉強をしよう。')

In [ ]:
# 4-5
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう。')

In [ ]:
# 4-6
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

In [ ]:
# 4-7
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう。')
print(input_ids)

In [ ]:
# 4-8
tokenizer.convert_ids_to_tokens(input_ids)

In [ ]:
# 4-9
text = '明日の天気は晴れだ。'
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

In [ ]:
# 4-10
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

In [ ]:
# 4-11
text_list = ['明日の天気は晴れだ。','パソコンが急に動かなくなった。']
tokenizer(
    text_list, max_length=10, padding='max_length', truncation=True
)

In [ ]:
# 4-12
tokenizer(text_list, padding='longest')

In [ ]:
# 4-13
tokenizer(
    text_list,
    max_length=10,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

In [ ]:
# 4-14
# モデルのロード
model_name = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

# BERTをGPUに載せる
bert = bert.cuda()

In [ ]:
# 4-15
print(bert.config)

In [ ]:
# 4-16
text_list = [
    '明日は自然言語処理の勉強をしよう。',
    '明日はマシーンラーニングの勉強をしよう。'
]

# 文章の符号化
encoding = tokenizer(
    text_list,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# データをGPUに載せる
encoding = { k: v.cuda() for k, v in encoding.items() }

# BERTでの処理
output = bert(**encoding) # それぞれの入力は2次元のtorch.Tensor
last_hidden_state = output.last_hidden_state # 最終層の出力

In [ ]:
# 4-17
output = bert(
    input_ids=encoding['input_ids'],
    attention_mask=encoding['attention_mask'],
    token_type_ids=encoding['token_type_ids']
)

In [ ]:
# 4-18
print(last_hidden_state.size()) #テンソルのサイズ

In [ ]:
# 4-19
with torch.no_grad():
    output = bert(**encoding)
    last_hidden_state = output.last_hidden_state

In [ ]:
# 4-20
last_hidden_state = last_hidden_state.cpu() # CPUにうつす。
last_hidden_state = last_hidden_state.numpy() # numpy.ndarrayに変換
last_hidden_state = last_hidden_state.tolist() # リストに変換